In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, LeaveOneGroupOut
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Concatenate, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm
import time
import warnings

# Change csv files to dataframes
def read_csv_files(directory):
    data_frames = {}
    annotations = {}
    print("\nReading CSV files:")
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            print(f"  Reading: {filename}")
            df = pd.read_csv(os.path.join(directory, filename))
            parts = filename.split('_')
            file_type = parts[0]
            participant = parts[-2] if len(parts) > 2 else 'unknown'
            session = parts[-1].split('.')[0] if len(parts) > 1 else 'unknown'
            if file_type == 'annotation':
                key = (participant, session)
                annotations[key] = df
            elif not file_type == 'E4 ACC data':
                key = (participant, session, file_type)
                data_frames[key] = df
            else:
                print(f"    Skipped: {filename} (ACC data)")
    print(f"Total files read: {len(data_frames)}")
    return data_frames, annotations

# Select useful data only (after game start, interpolate)
def preprocess_data(df, baseline_duration):
    if 'started' in df['game started'].values:
        start_idx = df[df['game started'] == 'started'].index[0]
        start_unix_time = df.loc[start_idx, 'UNIX Time']
        target_unix_time = start_unix_time - baseline_duration
        idx = (df['UNIX Time'] - target_unix_time).abs().idxmin()
        df = df.iloc[idx:]
    else:
        print("Warning: 'game start' marker not found. Using all data.")
        
    warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df[numeric_columns] = df[numeric_columns].interpolate(method='linear', limit_direction='both')
    
    return df

def z_score_normalize(df, baseline_duration):
    numeric_columns = df.select_dtypes(include=[np.number]).columns.drop('UNIX Time')
    baseline = df.iloc[:baseline_duration]
    mean = baseline[numeric_columns].mean()
    std = baseline[numeric_columns].std()
    std = std.replace(0, 1)  # Replace zero std with 1 to avoid division by zero
    normalized_df = df.copy()
    normalized_df[numeric_columns] = (df[numeric_columns] - mean) / std
    return normalized_df

def process_data(df, baseline_duration):
    df = preprocess_data(df, baseline_duration)
    
    if 'UNIX Time' not in df.columns or df['UNIX Time'].isnull().all():
        print("Error: 'UNIX Time' column not found or all values are null")
        return pd.DataFrame()
    
    game_start_time = df['UNIX Time'].min() + 5
    
    unix_time = df['UNIX Time']
    df_numeric = df.select_dtypes(include=[np.number]).drop(columns=['UNIX Time'])
    df_numeric['UNIX Time'] = unix_time
    
    normalized_df = z_score_normalize(df_numeric, baseline_duration)
    
    df_after_start = normalized_df[normalized_df['UNIX Time'] >= game_start_time]
    
    if df_after_start.empty:
        print(f"Warning: No data available after assumed game start")
        return pd.DataFrame(columns=['UNIX Time'])
    
    return df_after_start

def make_sequences(df, time_df, annotation, window_size):
    sequences = []
    challenge_sequences = []
    engagement_sequences = []
    for i in range(0, len(df) - window_size + 1):
        sequences.append(df.iloc[i:i+window_size].values)
        time = time_df.iloc[i]
        for j in range(0, len(annotation)):
            if annotation.loc[j, 'UNIX Start Time'] <= time and annotation.loc[j, 'UNIX End Time'] >= time:
                challenge_sequences.append(annotation.loc[j, 'Challenge'])
                engagement_sequences.append(annotation.loc[j, 'Engagement'])
                break
        else:
            challenge_sequences.append(1)
            engagement_sequences.append(1)
    return np.array(sequences), np.array(challenge_sequences), np.array(engagement_sequences)

def create_dataset(data_frames, annotations, window_size, expected_features, baseline_duration):
    X_data = []
    challenge_data = []
    engagement_data = []
    participant_data = []
    all_columns = set()
    
    for (participant, session, file_type), df in data_frames.items():
        processed_df = process_data(df, baseline_duration)
        processed_df = processed_df.drop(columns=['UNIX Time'])
        all_columns.update(processed_df.columns)
        
    all_columns.discard('game started')
    print(f'all_columns = {all_columns}')
    print(f'length of all_columns = {len(all_columns)}')
    
    all_columns = sorted(list(all_columns))[:expected_features] # Limit to expected number of features
    
    for (participant, session, file_type), df in data_frames.items():
        processed_df_unix = process_data(df, baseline_duration)
        processed_df = processed_df_unix.drop(columns=['UNIX Time'])
        
        for col in all_columns:
            if col not in processed_df.columns:
                processed_df[col] = 0
        
        processed_df = processed_df[all_columns]
        
        # Pad or truncate to match expected features
        if processed_df.shape[1] < expected_features:
            padding = pd.DataFrame(0, index=processed_df.index, 
                                   columns=[f'pad_{i}' for i in range(expected_features - processed_df.shape[1])])
            processed_df = pd.concat([processed_df, padding], axis=1)
        elif processed_df.shape[1] > expected_features:
            processed_df = processed_df.iloc[:, :expected_features]
            
        annotation = annotations[(participant, session)]
        print(participant, session)
        sequences, challenge_sequences, engagement_sequences = make_sequences(processed_df, processed_df_unix['UNIX Time'], annotation, window_size)
        if len(sequences) > 0:
            X_data.append(sequences)
            challenge_data.append(challenge_sequences)
            engagement_data.append(engagement_sequences)
            participant_data.extend([participant] * len(sequences))
    
    X = np.concatenate(X_data)
    challenge = np.concatenate(challenge_data)
    engagement = np.concatenate(engagement_data)
    participants = np.array(participant_data)
    
    return X, challenge, engagement, participants

def create_advanced_lstm_model(input_shape, n_classes, lstm_units, dense_units, dropout_rate):
    inputs = Input(shape=input_shape)
    
    low_level = TimeDistributed(Dense(32, activation='relu'))(inputs)
    low_level = TimeDistributed(Dense(16, activation='relu'))(low_level)
    
    high_level = LSTM(lstm_units[0], return_sequences=True)(inputs)
    high_level = LSTM(lstm_units[1])(high_level)
    
    combined = Concatenate()([tf.keras.layers.Flatten()(low_level), high_level])
    
    x = Dense(dense_units, activation='relu')(combined)
    x = Dropout(dropout_rate)(x)
    
    outputs = Dense(n_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

def create_multi_output_model_with_levels(input_shape, lstm_units, dense_units, dropout_rate):
    inputs = Input(shape=input_shape)

    # Low-level 특징 추출
    low_level = TimeDistributed(Dense(32, activation='relu'))(inputs)
    low_level = TimeDistributed(Dense(16, activation='relu'))(low_level)

    # High-level 특징 추출
    high_level = LSTM(lstm_units[0], return_sequences=True)(inputs)
    high_level = LSTM(lstm_units[1])(high_level)

    # Low-level과 High-level 결합
    combined = Concatenate()([tf.keras.layers.Flatten()(low_level), high_level])

    # Engagement 출력
    engagement_output = Dense(dense_units, activation='relu')(combined)
    engagement_output = Dropout(dropout_rate)(engagement_output)
    engagement_output = Dense(5, activation='softmax', name='engagement_output')(engagement_output)

    # Challenge 출력
    challenge_output = Dense(dense_units, activation='relu')(combined)
    challenge_output = Dropout(dropout_rate)(challenge_output)
    challenge_output = Dense(5, activation='softmax', name='challenge_output')(challenge_output)

    # 모델 생성
    model = Model(inputs=inputs, outputs=[engagement_output, challenge_output])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss={'engagement_output': 'categorical_crossentropy',
                        'challenge_output': 'categorical_crossentropy'},
                  metrics={'engagement_output': 'accuracy',
                           'challenge_output': 'accuracy'})
    
    return model

def loso_evaluation(X, y, participants, hyperparameters):
    logo = LeaveOneGroupOut()
    loso_scores = []
    
    y = np.stack((y[0], y[1]), axis=1)
    
    for i, (train_index, test_index) in enumerate(logo.split(X, y, participants)):
        print(f"\nLOSO Iteration {i+1}/{len(np.unique(participants))}")
        print(f"Test participant: {participants[test_index[0]]}")
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Validation split from the train set
        val_split_idx = int(0.8 * len(X_train))  # 80% train, 20% validation
        X_train, X_val = X_train[:val_split_idx], X_train[val_split_idx:]
        y_train, y_val = y_train[:val_split_idx], y_train[val_split_idx:]
        
        model = create_multi_output_model_with_levels(
            input_shape=(X_train.shape[1], X_train.shape[2]),
            lstm_units=hyperparameters['lstm_units'],
            dense_units=hyperparameters['dense_units'],
            dropout_rate=hyperparameters['dropout_rate']
        )
        
        y_train_engagement = to_categorical(y_train[:, 0])
        y_train_challenge = to_categorical(y_train[:, 1])
        
        y_val_engagement = to_categorical(y_val[:, 0])
        y_val_challenge = to_categorical(y_val[:, 1])
        
        y_test_engagement = to_categorical(y_test[:, 0])
        y_test_challenge = to_categorical(y_test[:, 1])
        
        # EarlyStopping 콜백 추가
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
        print("Training model...")
        history = model.fit(X_train, 
                            [y_train_engagement, y_train_challenge],
                            epochs=hyperparameters['epochs'], 
                            batch_size=hyperparameters['batch_size'],
                            validation_data=(X_val, [y_val_engagement, y_val_challenge]),
                            callbacks=[early_stopping], 
                            verbose=0)
        
        # 학습 그래프 그리기
        plt.figure(figsize=(12, 5))

        # Loss 그래프
        plt.subplot(1, 2, 1)
        plt.plot(history.history['engagement_output_loss'], label='Engagement Loss')
        plt.plot(history.history['challenge_output_loss'], label='Challenge Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title('Training and Validation Loss')
        plt.legend()

        # Accuracy 그래프
        plt.subplot(1, 2, 2)
        plt.plot(history.history['engagement_output_accuracy'], label='Engagement Accuracy')
        plt.plot(history.history['challenge_output_accuracy'], label='Challenge Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.title('Training and Validation Accuracy')
        plt.legend()

        plt.tight_layout()
        plt.show()
        
        print("Evaluating model...")
        y_pred = model.predict(X_test)
        y_pred_engagement = np.argmax(y_pred[0], axis=1)
        y_pred_challenge = np.argmax(y_pred[1], axis=1)
        
        y_test_engagement_classes = np.argmax(y_test_engagement, axis=1)
        y_test_challenge_classes = np.argmax(y_test_challenge, axis=1)
        
        accuracy_engagement = accuracy_score(y_test_engagement_classes, y_pred_engagement)
        accuracy_challenge = accuracy_score(y_test_challenge_classes, y_pred_challenge)
        loso_scores.append((accuracy_engagement + accuracy_challenge) / 2)
        print(f"Accuracy for participant {participants[test_index[0]]}:")
        print(f" - Engagement: {accuracy_engagement:.4f}")
        print(f" - Challenge: {accuracy_challenge:.4f}")
        
        print("Classification Report for Engagement Output:")
        print(classification_report(y_test_engagement_classes, y_pred_engagement, target_names=['Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5']))
        
        print("Classification Report for Challenge Output:")
        print(classification_report(y_test_challenge_classes, y_pred_challenge, target_names=['Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5']))
        
        # Confusion Matrix for Engagement
        cm_engagement = confusion_matrix(y_test_engagement_classes, y_pred_engagement)
        disp_engagement = ConfusionMatrixDisplay(confusion_matrix=cm_engagement, display_labels=['Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5'])
        disp_engagement.plot(cmap=plt.cm.Blues)
        plt.title(f'Confusion Matrix for Engagement Output (Participant {participants[test_index[0]]})')
        plt.show()

        # Confusion Matrix for Challenge
        cm_challenge = confusion_matrix(y_test_challenge_classes, y_pred_challenge)
        disp_challenge = ConfusionMatrixDisplay(confusion_matrix=cm_challenge, display_labels=['Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5'])
        disp_challenge.plot(cmap=plt.cm.Blues)
        plt.title(f'Confusion Matrix for Challenge Output (Participant {participants[test_index[0]]})')
        plt.show()
    
    return np.mean(loso_scores), np.std(loso_scores)

def stratified_kfold_evaluation(X, y, hyperparameters):
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    
    fold_accuracies = []
    
    for i, (train_index, test_index) in enumerate(skf.split(X, y[0])):
        print(f"\nStratified K-Fold Iteration {i+1}/{skf.n_splits}")
        
        X_train, X_test = X[train_index], X[test_index]
        y_train_challenge, y_test_challenge = y[0][train_index], y[0][test_index]  # Challenge
        y_train_engagement, y_test_engagement = y[1][train_index], y[1][test_index]
        
        # Apply SMOTE only on the training data
        print("Applying SMOTE to the training data...")
        smote = SMOTE(random_state=42)
        X_train_flat = X_train.reshape(X_train.shape[0], -1)
        X_train_resampled_challenge, y_train_resampled_challenge = smote.fit_resample(X_train_flat, y_train_challenge)
        X_train_resampled_challenge = X_train_resampled_challenge.reshape(-1, X_train.shape[1], X_train.shape[2])  # 원래의 3D 형태로 복원
        print(f"After SMOTE - Challenge training data shape: {X_train_resampled_challenge.shape}, Challenge training labels shape: {y_train_resampled_challenge.shape}")
        
        # Engagement 레이블에 대해 SMOTE 적용
        print("Applying SMOTE to the training data (Engagement)...")
        X_train_resampled_engagement, y_train_resampled_engagement = smote.fit_resample(X_train_flat, y_train_engagement)
        X_train_resampled_engagement = X_train_resampled_engagement.reshape(-1, X_train.shape[1], X_train.shape[2])  # 원래의 3D 형태로 복원
        print(f"After SMOTE - Engagement training data shape: {X_train_resampled_engagement.shape}, Engagement training labels shape: {y_train_resampled_engagement.shape}")
        
        # Challenge와 Engagement 레이블의 길이를 맞추기 위해 두 레이블의 샘플 수를 동일하게 설정
        min_length = min(len(y_train_resampled_challenge), len(y_train_resampled_engagement))
        X_train_resampled_challenge = X_train_resampled_challenge[:min_length]
        X_train_resampled_engagement = X_train_resampled_engagement[:min_length]
        y_train_resampled_challenge = y_train_resampled_challenge[:min_length]
        y_train_resampled_engagement = y_train_resampled_engagement[:min_length]
        
        # Encode labels (using the pre-fitted encoder)
        y_train_cat_challenge = to_categorical(y_train_resampled_challenge)
        y_test_cat_challenge = to_categorical(y_test_challenge)
        
        y_train_cat_engagement = to_categorical(y_train_resampled_engagement)
        y_test_cat_engagement = to_categorical(y_test_engagement)
        
        # EarlyStopping callback
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
        # Create and train the model
        model = create_multi_output_model_with_levels(
            input_shape=(X_train_resampled_challenge.shape[1], X_train_resampled_challenge.shape[2]),
            lstm_units=hyperparameters['lstm_units'],
            dense_units=hyperparameters['dense_units'],
            dropout_rate=hyperparameters['dropout_rate']
        )
        
        print("Training model...")
        history = model.fit(
            X_train_resampled_challenge, [y_train_cat_challenge, y_train_cat_engagement],
            epochs=hyperparameters['epochs'],
            batch_size=hyperparameters['batch_size'],
            validation_data=(X_test, [y_test_cat_challenge, y_test_cat_engagement]),
            verbose=1  # Set to 1 for more detailed output
        )
        
        # Plot training history
        plt.figure(figsize=(12, 5))

        # Loss plot
        plt.subplot(1, 2, 1)
        plt.plot(history.history['challenge_output_loss'], label='Challenge Output Loss (Training)')
        plt.plot(history.history['val_challenge_output_loss'], label='Challenge Output Loss (Validation)')
        plt.plot(history.history['engagement_output_loss'], label='Engagement Output Loss (Training)')
        plt.plot(history.history['val_engagement_output_loss'], label='Engagement Output Loss (Validation)')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title(f'Fold {i+1} - Training and Validation Loss')
        plt.legend()

        # Accuracy plot
        plt.subplot(1, 2, 2)
        plt.plot(history.history['engagement_output_accuracy'], label='Training Engagement Accuracy')
        plt.plot(history.history['val_engagement_output_accuracy'], label='Validation Engagement Accuracy')
        plt.plot(history.history['challenge_output_accuracy'], label='Training Challenge Accuracy')
        plt.plot(history.history['val_challenge_output_accuracy'], label='Validation Challenge Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.title(f'Fold {i+1} - Training and Validation Accuracy')
        plt.legend()

        plt.tight_layout()
        plt.show()
        
        # Evaluate the model
        print("Evaluating model on validation data...")
        y_pred = model.predict(X_test)
        y_pred_challenge_classes = np.argmax(y_pred[0], axis=1)
        y_pred_engagement_classes = np.argmax(y_pred[1], axis=1)
        y_test_challenge_classes = np.argmax(y_test_cat_challenge, axis=1)
        y_test_engagement_classes = np.argmax(y_test_cat_engagement, axis=1)
        
        challenge_accuracy = accuracy_score(y_test_challenge_classes, y_pred_challenge_classes)
        engagement_accuracy = accuracy_score(y_test_engagement_classes, y_pred_engagement_classes)
        fold_accuracies.append((challenge_accuracy, engagement_accuracy))
        print(f"Challenge Accuracy for fold {i+1}: {challenge_accuracy:.4f}")
        print(f"Engagement Accuracy for fold {i+1}: {engagement_accuracy:.4f}")
        
        print("Classification Report for Challenge Output:")
        print(classification_report(y_test_challenge_classes, y_pred_challenge_classes))
        
        print("Classification Report for Engagement Output:")
        print(classification_report(y_test_engagement_classes, y_pred_engagement_classes))
        
        
        cm_challenge = confusion_matrix(y_test_challenge_classes, y_pred_challenge_classes)
        disp_challenge = ConfusionMatrixDisplay(confusion_matrix=cm_challenge)
        disp_challenge.plot(cmap=plt.cm.Blues)
        plt.title(f'Challenge Confusion Matrix for fold {i+1}')
        plt.show()

        # Confusion Matrix for Engagement
        cm_engagement = confusion_matrix(y_test_engagement_classes, y_pred_engagement_classes)
        disp_engagement = ConfusionMatrixDisplay(confusion_matrix=cm_engagement)
        disp_engagement.plot(cmap=plt.cm.Blues)
        plt.title(f'Engagement Confusion Matrix for fold {i+1}')
        plt.show()
    
    return np.mean(fold_accuracies), np.std(fold_accuracies)

def main():
    directory = r'C:\Users\USER\Downloads\pilotdata'
    data_frames, annotations = read_csv_files(directory)
    
    window_size = 10  # 10-second window
    n_classes = 5  # Only 5-class classification
    expected_features = 28  # Set this to the number of features you expect
    baseline_duration = 30  # Set this to your baseline duration in seconds
    
    print(f"\n{'='*50}")
    print(f"Running 5-class classification with Loso evaluation")
    print(f"{'='*50}")
    
    X, challenge, engagement, participants = create_dataset(data_frames, annotations, window_size, expected_features, baseline_duration)
    
    if X is None or challenge is None or engagement is None or participants is None:
        print("Failed to create dataset.")
        return
    
    print(f"Dataset shape: {X.shape}")
    print(f"Number of sequences: {X.shape[0]}")
    print(f"Sequence length: {X.shape[1]}")
    print(f"Number of features: {X.shape[2]}")
    print(f"Class distribution: {np.unique(challenge, return_counts=True)}")
    print(f"Class distribution: {np.unique(engagement, return_counts=True)}")
    
    # Encode labels once
    le_challenge = LabelEncoder()
    le_challenge.fit(challenge)  # Fit on the entire challenge dataset to ensure consistency
    challenge_encoded = le_challenge.transform(challenge)
    
    le_engagement = LabelEncoder()
    le_engagement.fit(engagement)  # Fit on the entire engagement dataset to ensure consistency
    engagement_encoded = le_engagement.transform(engagement)
    
    print("\nAfter Label Encoding:")
    print(f"Challenge classes: {le_challenge.classes_}")
    print(f"Encoded challenge labels: {np.unique(challenge_encoded, return_counts=True)}")
    print(f"Engagement classes: {le_engagement.classes_}")
    print(f"Encoded engagement labels: {np.unique(engagement_encoded, return_counts=True)}")

    
    # Use the best hyperparameters you provided
    best_hyperparameters = {
        'lstm_units': (32, 16),
        'dense_units': 64,
        'dropout_rate': 0.3,
        'epochs': 50,
        'batch_size': 256
    }
    
    print(f'challenge_encoded : {challenge_encoded.shape}')
    print(f'engagement_encoded : {engagement_encoded.shape}')
    
    try:
        print("\nStarting Loso evaluation with best hyperparameters...")
        loso_mean, loso_std = loso_evaluation(X, [challenge_encoded, engagement_encoded], participants, best_hyperparameters)
        print(f"\nLoso evaluation results:")
        print(f"Mean accuracy: {loso_mean:.4f} (+/- {loso_std:.4f})")
        
        # Optionally, train a final model on the entire dataset
        # Apply SMOTE to the entire dataset (if desired), but be cautious of data leakage
        # Here, we'll skip SMOTE for the final model to avoid potential leakage
        final_model = create_multi_output_model_with_levels(
            input_shape=(X.shape[1], X.shape[2]),
            lstm_units=best_hyperparameters['lstm_units'],
            dense_units=best_hyperparameters['dense_units'],
            dropout_rate=best_hyperparameters['dropout_rate']
        )
        
        y_cat = [to_categorical(challenge_encoded, num_classes=n_classes), 
                 to_categorical(engagement_encoded, num_classes=n_classes)]
        
        print("\nTraining final model on all data...")
        early_stopping_final = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        final_model.fit(X, y_cat, epochs=best_hyperparameters['epochs'], 
                       batch_size=best_hyperparameters['batch_size'], 
                       callbacks=[early_stopping_final], verbose=1)
        
        # Save the final model
        model_filename = r"C:\Users\USER\Downloads\pilotdata\best_lstm_model_5class1127.h5"
        final_model.save(model_filename)
        print(f"\nFinal model saved as {model_filename}")
        
        # Save the label encoder
        import joblib
        le_filename = r"C:\Users\USER\Downloads\pilotdata\label_encoder_5class1127.joblib"
        joblib.dump([le_challenge, le_engagement], le_filename)
        print(f"Label encoder saved as {le_filename}")
        
    except Exception as e:
        import traceback
        print(f"An error occurred during the Loso evaluation or model saving:")
        print(str(e))
        print("Traceback:")
        traceback.print_exc()

if __name__ == "__main__":
    main()


Reading CSV files:
  Reading: annotation_cy_1.csv
  Reading: annotation_cy_2.csv
  Reading: annotation_cy_3.csv
  Reading: annotation_cy_4.csv
  Reading: annotation_cy_5.csv
  Reading: annotation_db_1.csv
  Reading: annotation_db_2.csv
  Reading: annotation_db_3.csv
  Reading: annotation_db_4.csv
  Reading: annotation_db_5.csv
  Reading: annotation_es_1.csv
  Reading: annotation_es_2.csv
  Reading: annotation_es_3.csv
  Reading: annotation_es_4.csv
  Reading: annotation_es_5.csv
  Reading: annotation_gb_1.csv
  Reading: annotation_gb_2.csv
  Reading: annotation_gb_3.csv
  Reading: annotation_gb_4.csv
  Reading: annotation_gb_5.csv
  Reading: annotation_jw_1.csv
  Reading: annotation_jw_2.csv
  Reading: annotation_jw_3.csv
  Reading: annotation_jw_4.csv
  Reading: annotation_jw_5.csv
  Reading: annotation_kb_1.csv
  Reading: annotation_kb_2.csv
  Reading: annotation_kb_3.csv
  Reading: annotation_kb_4.csv
  Reading: annotation_kb_5.csv
  Reading: annotation_mw_1.csv
  Reading: annotati

KeyboardInterrupt: 